# Customers Who Bought This Frequently Buy This!
In this notebook we will explore which items were frequently purchased together. Using this information, we can predict which items a customer will buy after we observe what they have already bought!

In [1]:
import numpy as np
import pandas as pd
import gc
import os
import time
import random
from tqdm.auto import tqdm
import datetime

# Load Tranactions

In [2]:
#log_date = '2020-09-16'
log_date = '2020-09-09'
Ntop = 2
#ch = 'on'
ch = 'off'
#ch = ''

In [3]:
# parameter injection cell

In [4]:
# LOAD TRANSACTIONS DATAFRAME
df = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
df['t_dat'] = pd.to_datetime(df['t_dat'])
df = df.query(f"t_dat < '{log_date}'").reset_index(drop=True)
df

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2
...,...,...,...,...,...
31292767,2020-09-08,fff2282977442e327b45d8c89afde25617d00124d0f999...,919786002,0.042356,2
31292768,2020-09-08,fffbdd2f8e59d45c0fb50a14b0ea555f7daa5307b3f922...,765308002,0.033881,2
31292769,2020-09-08,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,689365050,0.010017,2
31292770,2020-09-08,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,884081001,0.012898,2


In [7]:
# online/offline or all
Nth = 100
if ch == 'off':
    df = df.query('sales_channel_id == 1')
    Nch = 25
elif ch == 'on':
    df = df.query('sales_channel_id == 2')
    Nch = 75

In [8]:
# REDUCE MEMORY OF DATAFRAME
df = df[['customer_id','article_id']]
df['customer_id'] = df['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')
df['article_id'] = df['article_id'].astype('int32')
_ = gc.collect()

# Find Items Purchased Together
We will use RAPID cuDF to speed up the dataframe search commands below

In [ ]:
# FIND ITEMS PURCHASED TOGETHER
vc = df.article_id.value_counts()
# 少なすぎるものは削る。vc>50でだいたい半分くらいになる
vc = vc[vc>Nch]
vc

In [ ]:
# FIND ITEMS PURCHASED TOGETHER
pairs_np = {} # cudfでなくても読める用
for j,i in enumerate(vc.index.values):
#for j,i in enumerate(vc.index.values[:100]):
    if j%100==0: print(j,', ',end='')
    USERS = df.loc[df.article_id==i.item(),'customer_id'].unique() # 特定のarticleを買っているcustomer
    vc2 = df.loc[(df.customer_id.isin(USERS))&(df.article_id!=i.item()),'article_id'].value_counts() # 自分自身を除く、同時に買われたarticle
    pairs_np[i.item()] = vc2.index[:Ntop]

In [ ]:
sum_pair = len(pairs_np)
s = 0
for v in pairs_np.values():
    s += len(v)
print(s/sum_pair) # mapの長さの平均値

In [ ]:
#np.save(f'pairs_np_{log_date}_{Ntop}.npy', pairs_np)

In [ ]:
import pickle
    
with open(f'pairs_np_{log_date}_{Ntop}{ch}.pkl', "wb") as f:
    pickle.dump(pairs_np,f)
